# Tempo de processamento com e sem PCA

Esse jupyter foi construido remotamente e deve rodar na Raspberry. A ideia é repetir um experimento que foi executado na servidora do laboratório para medir o tempo de processamento para classificar os dados de teste com e sem a utilização de PCA. Os testes na servidora indicaram que o PCA dobrou o tempo de processamento, pois transformar os dados de teste para uma dimensionalidade mais baixa demorava mais do que simplesmente classificar logo em todas as dimensões originais.

Nessa altura já descobrimos através do Feature Importance (Índice de GINI) que podemos deixar de extrair as 135 features para extrair apenas 58 que o resultado em acurácia não é afetado. Também já temos o modelo de classificador mais apropriado em termos de tempo de processamento e acurácia (SVM com SGD). Esse classificador foi o vencedor tanto no artigo do NUVEM quanto do SBrT. 

In [1]:
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
import time
import pandas as pd
import numpy as np

In [2]:
def ajustarDataset(caminhoCSV, pasta):
    
    # ABRINDO
    print("Carregando CSV com o dataset")
    dataframe = pd.read_csv(caminhoCSV)
    print("CSV carregado")
    
    # SEPARANDO DATA, TARGET E O NOME DOS ARQUIVOS
    print("Separando o que é X e Y")
    data     = dataframe.iloc[:, 1:-1].to_numpy()
    target   = dataframe.iloc[:, -1].to_numpy()
    arquivos = dataframe.iloc[:, 0].to_numpy()
    
    # LIMPANDO A MEMORIA
    print("Limpando o dataframe pandas da memória")
    del dataframe
    
    # SEPARANTE TREINO E TESTE PELO NOME DO ARQUIVO (O ULTIMO NUMERO NO NOME E A PASTA casual_000_1.wav)
    print("Começando a separar o que é treinamento e teste")
    xTrain, xTest, yTrain, yTest = [], [], [], []
    for xAtual, yAtual, arquivoAtual in zip(data, target, arquivos):
        
        # SE FOR DA PASTA ESCOLHIDA VAI PRO TESTE
        if int(arquivoAtual.split("_")[2][0]) == pasta:
            xTest.append(xAtual)
            yTest.append(yAtual)
        
        # SENAO VAI PRO TREINAMENTO
        else:
            xTrain.append(xAtual)
            yTrain.append(yAtual)
    print("Separação finalizada")   
    
    # PASSANDO TUDO PRA NUMPY
    xTrain = np.array(xTrain)
    xTest  = np.array(xTest)
    yTrain = np.array(yTrain)
    yTest  = np.array(yTest)
            
    # VERIFICANDO A DIMENSIONALIDADE
    print("Garantindo que a dimensionalidade está correta")
    assert xTrain.shape[0] + xTest.shape[0] == data.shape[0], "Dimensionalidade dos Xs não bate."
    assert yTrain.shape[0] + yTest.shape[0] == target.shape[0], "Dimensionalidade dos Ys não bate."
    assert xTrain.shape[1] == xTest.shape[1] and xTest.shape[1] == data.shape[1], "Número de features não bate."
    
    # VAMO QUE VAMO
    print("Função ajustarDataset() finalizada")
    return xTrain, xTest, yTrain, yTest

In [3]:
def medirTempoComESemPCA(xTest, objClassificador, objPCA):
    
    print("Iniciando a medição de tempo")
    tempoInicio = time.time()
    
    if objPCA != None:
        print("Aplicando o PCA")
        xTest = objPCA.transform(xTest)
        print("PCA aplicado")
        
    print("Realizando a predição de xTest")
    __ = objClassificador.predict(xTest)
    print("Predição finalizada")
    
    tempoFim = time.time()
    print("Parando de medir o tempo")
    
    return tempoFim - tempoInicio

In [4]:
# CAMINHO PARA O CSV COM 58 FEATURES
caminhoCSV = "/home/pi/Datasets/SESA_v2_16kHz_16bits_58features.csv"

# SELECIONANDO UMA UNICA ITERACAO DO K FOLD
xTrain, xTest, yTrain, __ = ajustarDataset(caminhoCSV, pasta=1)

# TREINANDO UM CLASSIFICADOR COM TODAS AS DIMENSOES
print("Inciando o treinamento do classificador em 58D")
objClassificadorSemPCA = SGDClassifier(alpha=0.0001, average=False, class_weight=None, early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000, n_iter_no_change=5, n_jobs=1, penalty='l2', power_t=0.5, random_state=None, shuffle=True, tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)
objClassificadorSemPCA.fit(xTrain, yTrain)
print("objClassificadorSemPCA treinado com xTrain de", xTrain.shape)

# TREINANDO COM AS DIMENSOES REDUZIDAS
print("Inciando o fit do objeto PCA")
objPCA = PCA(n_components=38)
objPCA.fit(xTrain)
xTrain = objPCA.transform(xTrain)
print("Objeto PCA pronto")
print("Inciando o treinamento do classificador em 38D")
objClassificadorComPCA = SGDClassifier(alpha=0.0001, average=False, class_weight=None, early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000, n_iter_no_change=5, n_jobs=1, penalty='l2', power_t=0.5, random_state=None, shuffle=True, tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)
objClassificadorComPCA.fit(xTrain, yTrain)
print("objClassificadorComPCA treinado com xTrain de", xTrain.shape)

# LIMPANDO A MEMORIA POIS SO VOU PRECISAR DOS OBJETOS DOS CLASSIFICADORES JA TREINADOS, DO OBJ PCA E DO XTEST
print("Removendo xTrain e yTrain da memória")
del xTrain
del yTrain

Carregando CSV com o dataset
CSV carregado
Separando o que é X e Y
Limpando o dataframe pandas da memória
Começando a separar o que é treinamento e teste
Separação finalizada
Garantindo que a dimensionalidade está correta
Função ajustarDataset() finalizada
Inciando o treinamento do classificador em 58D
objClassificadorSemPCA treinado com xTrain de (21902, 58)
Inciando o fit do objeto PCA
Objeto PCA pronto
Inciando o treinamento do classificador em 38D
objClassificadorComPCA treinado com xTrain de (21902, 38)
Removendo xTrain e yTrain da memória


In [8]:
# RODANDO SEM PCA
repetibilidade = 120
arrayTemposSemPCA = Parallel(n_jobs=-1, verbose=10)(delayed(medirTempoComESemPCA)(xTest, objClassificadorSemPCA, None) for i in range(repetibilidade))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0911s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1985s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 120 | elapsed:    1.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    1.7s finished


In [9]:
# RODANDO COM PCA
repetibilidade = 120
arrayTemposComPCA = Parallel(n_jobs=-1, verbose=10)(delayed(medirTempoComESemPCA)(xTest, objClassificadorComPCA, objPCA) for i in range(repetibilidade))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1671s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    4.5s finished


In [10]:
print("Tempo total médio sem PCA:", np.mean(arrayTemposSemPCA), "+-", np.std(arrayTemposSemPCA))
print("Tempo total médio com PCA:", np.mean(arrayTemposComPCA), "+-", np.std(arrayTemposComPCA))

Tempo total médio sem PCA: 0.0317365030447642 +- 0.007211407879928582
Tempo total médio com PCA: 0.12620155612627665 +- 0.018542676372179878


**Resultado ANTERIOR (SERVIDORA):**

Tempo total médio sem PCA: 0.002251005172729492 +- 0.0005577909513149635

Tempo total médio com PCA: 0.004084736108779907 +- 0.000670405992919316

**Resultado ATUAL (RASPBERRY):**

Tempo total médio sem PCA: 0.0317365030447642 +- 0.007211407879928582

Tempo total médio com PCA: 0.12620155612627665 +- 0.018542676372179878

**Considerações:**

Os testes foram realizados na Raspberry em que cada rodada considerou apenas o tempo para:

- Aplicar (ou não) o PCA
    
- Classificar todas as amostras de teste

Cada rodada usou apenas um núcleo do processador. NOVAMENTE teste foi repetido 120x para os dois casos (com e sem PCA) para garantir veracidade. O treinamento dos classificadores e o treinamento do PCA não afetou o tempo medido pois essas etapas de "fit" foram feitas anteriormente. O cuidado foi para garantir que apenas o tempo da aplicação ou não do PCA + a classificação de novas amostras fosse medido.

Mas quanto tempo a raspberry demorou pra classificar cada janela, em média?

In [15]:
print("Quantidade de amostras de teste:", xTest.shape[0])
print("Tempo médio para classificar uma única amostra (janela de 200ms):", np.mean(arrayTemposSemPCA)/xTest.shape[0])

Quantidade de amostras de teste: 5486
Tempo médio para classificar uma única amostra (janela de 200ms): 5.7849987321845055e-06


**Resultado:**
    
Tempo médio para classificar uma única amostra (janela de 200ms) com a Raspberry: 5.7849987321845055e-06